In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


# Load your demand data into a DataFrame
data = pd.read_csv('ProductDemand.csv')
data



,ID,Store ID,Total Price,Base Price,Units Sold
0,1,8091,99.0375,111.8625,20
1,2,8091,99.0375,99.0375,28
2,3,8091,133.9500,133.9500,19
3,4,8091,133.9500,133.9500,44
4,5,8091,141.0750,141.0750,52
...,...,...,...,...,...
150145,212638,9984,235.8375,235.8375,38
150146,212639,9984,235.8375,235.8375,30
150147,212642,9984,357.6750,483.7875,31
150148,212643,9984,141.7875,191.6625,12


In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler



In [11]:
# Assuming you have features like price, promotions, and lagged demand
# Perform feature engineering and add these features to your DataFrame
data['ID'] = data['Store ID'].shift(1)
data['	Total Price'] = data['Base Price'] - data['Units Sold'].shift(1)
data['Total Price'] = (data['Base Price'] > 0).astype(int)
data

,ID,Store ID,Total Price,Base Price,Units Sold,\tTotal Price
1,NaN,8091,1,99.0375,28,NaN
2,8091.0,8091,1,133.9500,19,105.9500
3,8091.0,8091,1,133.9500,44,114.9500
4,8091.0,8091,1,141.0750,52,97.0750
5,8091.0,8091,1,227.2875,18,175.2875
...,...,...,...,...,...,...
150145,9984.0,9984,1,235.8375,38,212.8375
150146,9984.0,9984,1,235.8375,30,197.8375
150147,9984.0,9984,1,483.7875,31,453.7875
150148,9984.0,9984,1,191.6625,12,160.6625


In [14]:
# Define your feature set and target variable
X = data[['ID', 'Store ID',	'Total Price',	'Base Price']]
X

,ID,Store ID,Total Price,Base Price
1,NaN,8091,1,99.0375
2,8091.0,8091,1,133.9500
3,8091.0,8091,1,133.9500
4,8091.0,8091,1,141.0750
5,8091.0,8091,1,227.2875
...,...,...,...,...
150145,9984.0,9984,1,235.8375
150146,9984.0,9984,1,235.8375
150147,9984.0,9984,1,483.7875
150148,9984.0,9984,1,191.6625


In [15]:
y = data['Units Sold']
y

1         28
2         19
3         44
4         52
5         18
          ..
150145    38
150146    30
150147    31
150148    12
150149    15
Name: Units Sold, Length: 150149, dtype: int64

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [19]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test
X_train

array([[        nan, -1.8006192 ,  0.        , -1.0831272 ],
       [-1.80061327, -1.8006192 ,  0.        , -0.76652021],
       [-1.80061327, -1.8006192 ,  0.        , -0.76652021],
       ...,
       [ 1.27452513,  1.27451121,  0.        ,  2.27678369],
       [ 1.27452513,  1.27451121,  0.        , -0.43699049],
       [ 1.27452513,  1.27451121,  0.        , -0.24314948]])

In [20]:
# Initialize and train the XGBoost model
model = XGBRegressor(n_estimators=100, max_depth=3)
model.fit(X_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
# Make predictions
y_pred = model.predict(X_test)
y_pred

array([15.912153, 21.559523, 28.833385, ..., 23.247751, 71.96496 ,
       51.01367 ], dtype=float32)

In [28]:
# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 46.75602628558943
